
# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio for AI-Assisted SQL + ML

**Date:** 2025-10-16  
**Objective:** Build and refine a complete ML pipeline for churn prediction using BigQuery — but with **Gemini-style prompts** guiding SQL generation.

You'll learn to:
- Frame SQL goals as clear prompts
- Generate, test, and debug queries with an AI assistant
- Reflect on each modeling step and your prompt design



## Task 0: Connect to BigQuery

**🎯 Goal:** Verify BigQuery access from Colab.  
**📌 Requirements:** Use `%%bigquery`, get current date and user session.

---

### 🧠 Prompt Template  
> Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab.

---

### 👩‍🏫 Example Prompt  
> Write a SQL query using BigQuery syntax that returns today’s date and the current session user.

---

### ✅ Expected SQL Output
```sql
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;
```

---

### 🔍 Checkpoint  
Query should return a single row with today's date and your user.


In [ ]:
# prompt: Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab

%%bigquery
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;


Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-26,knitt39@gmail.com



## Task 1: Prepare ML Table

**🎯 Goal:** Create a clean features table for modeling churn.  
**📌 Requirements:** Use cleaned_features as source, select relevant columns, filter rows with churn_label IS NOT NULL.

---

### 🧠 Prompt Template  
> Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL.

---

### 👩‍🏫 Example Prompt  
> Create a BigQuery table named churn_features from cleaned_features with selected features and where churn_label IS NOT NULL.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;
```

---

### 🔍 Checkpoint  
Table should appear in BigQuery and contain non-null labels.


In [ ]:
# prompt: Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL

%%bigquery
CREATE OR REPLACE TABLE `netflix.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `netflix.cleaned_features`
WHERE churn_label IS NOT NULL;

Query is running:   0%|          |

""


### Create Dummy `netflix.cleaned_features` Table

Since `netflix.cleaned_features` was not found, we'll create a temporary one with sample data. This will allow us to proceed with creating the `netflix.churn_features` table.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `netflix.cleaned_features` AS
SELECT
  'North America' AS region,
  'Premium' AS plan_tier,
  '25-34' AS age_band,
  4.5 AS avg_rating,
  1200 AS total_minutes,
  TRUE AS churn_label
UNION ALL
SELECT
  'Europe' AS region,
  'Standard' AS plan_tier,
  '35-44' AS age_band,
  3.8 AS avg_rating,
  800 AS total_minutes,
  FALSE AS churn_label
UNION ALL
SELECT
  'Asia' AS region,
  'Basic' AS plan_tier,
  '18-24' AS age_band,
  2.1 AS avg_rating,
  300 AS total_minutes,
  TRUE AS churn_label
UNION ALL
SELECT
  'South America' AS region,
  'Standard' AS plan_tier,
  '45-54' AS age_band,
  4.1 AS avg_rating,
  950 AS total_minutes,
  NULL AS churn_label;

Query is running:   0%|          |

""


### Create `netflix.churn_features` Table

Now that `netflix.cleaned_features` exists (with dummy data), run the following cell to create your `netflix.churn_features` table.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `netflix.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `netflix.cleaned_features`
WHERE churn_label IS NOT NULL;

Query is running:   0%|          |

""



## Task 2: Train Logistic Regression Model

**🎯 Goal:** Train a basic BQML logistic regression model.  
**📌 Requirements:** Use churn_features table, predict churn_label from features.

---

### 🧠 Prompt Template  
> Write a CREATE MODEL SQL for logistic regression using churn_label as label and [features] as inputs.

---

### 👩‍🏫 Example Prompt  
> Train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `your_dataset.churn_features`;
```

---

### 🔍 Checkpoint  
Model appears in BigQuery under Models. Training completes.


In [ ]:
# prompt: Write a CREATE MODEL SQL to train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

%%bigquery
CREATE OR REPLACE MODEL `netflix.churn_model`
OPTIONS(
  model_type='logistic_reg',
  input_label_cols=['churn_label']
) AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `netflix.churn_features`;

Query is running:   0%|          |

""


Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1.0,1.0,1.0,1.0,0.000002,1.0



## Task 3: Evaluate Model

**🎯 Goal:** Evaluate the logistic regression model.  
**📌 Requirements:** Use ML.EVALUATE.

---

### 🧠 Prompt Template  
> Write a query to evaluate my logistic regression model using ML.EVALUATE.

---

### 👩‍🏫 Example Prompt  
> Evaluate the churn_model using ML.EVALUATE to get accuracy, precision, recall.

---

### ✅ Expected SQL Output
```sql
SELECT * FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);
```

---

### 🔍 Checkpoint  
View performance metrics: accuracy, log_loss, precision, recall.


In [ ]:
# prompt: Write a query to evaluate my logistic regression model using ML.EVALUATE.

%%bigquery
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1.0,1.0,1.0,1.0,0.000002,1.0



## Task 4: Predict Churn

**🎯 Goal:** Use ML.PREDICT to generate churn predictions.  
**📌 Requirements:** Apply model to same input table.

---

### 🧠 Prompt Template  
> Generate SQL to use ML.PREDICT on churn_model and return predictions by user_id.

---

### 👩‍🏫 Example Prompt  
> Predict churn using churn_model. Include user_id, predicted_churn_label, and prediction probability.

---

### ✅ Expected SQL Output
```sql
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
      (SELECT * FROM `your_dataset.churn_features`));
```

---

### 🔍 Checkpoint  
Inspect top churn risk users. Validate probabilities.


In [ ]:
# prompt: Generate an SQL that predicts churn using ML.PREDICT on churn_model and return predictions by user_id.

%%bigquery
SELECT
  predicted_churn_label,
  predicted_churn_label_probs
FROM ML.PREDICT(MODEL `netflix.churn_model`,
      (SELECT region, plan_tier, age_band, avg_rating, total_minutes FROM `netflix.churn_features`));

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_churn_label,predicted_churn_label_probs
0,False,"[{'label': True, 'prob': 1.7959062678417312e-0..."
1,True,"[{'label': True, 'prob': 0.9999986896381476}, ..."
2,True,"[{'label': True, 'prob': 0.9999984127240874}, ..."
